# Azure Cognitive Services で AI プログラミング

## 0. 準備 - Cognitive Services 接続情報を確認

このハンズオンでは Azure Cognitive Services で提供されている **事前構築済みの AI** を利用して AI 開発を体験します。

最初に Cognitive Services への接続情報が正しく設定されているかを確認します。

> 集合形式のハンズオンではあらかじめ接続情報が設定されているかもしれません。  
> その場合は、以下のセルを実行すると、エンドポイントとキーの値が表示されます。
>
> 接続情報が設定されていない場合は、 [**Cognitive リソースの作成**](./a01_createcog.ipynb) および [**ハンズオンの環境構築**](./0_setup.ipynb) を実施する必要があります。

In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
# Cognitive Services 利用のために Endpoint と Key とを読み込みます
load_dotenv()
cog_endpoint = os.getenv('COG_SERVICE_ENDPOINT')
cog_key = os.getenv('COG_SERVICE_KEY')

In [ ]:
print("Endpoint: " + cog_endpoint)
print("Key: " + cog_key)

---

## 1. 画像分析

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
# 画像のパスを決定
image_path = os.path.join('data', 'image', 'image1.jpg')
#image_path = os.path.join('data', 'image', 'image2.jpg')

In [ ]:
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

In [ ]:
# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

In [ ]:
image_stream = open(image_path, "rb")

features = ['Description', 'Tags', 'Adult', 'Objects', 'Faces']
#analysis = computervision_client.analyze_image_in_stream(image_stream, visual_features=features)
analysis = computervision_client.analyze_image_in_stream(image_stream, visual_features=features, language="ja")

In [ ]:
for tag in analysis.tags:
    print(f'{tag.name} ({(tag.confidence * 100):.1f}%)')

In [ ]:
import matplotlib.patches as patches

plt.axis('off')
plt.imshow(img)

ax = plt.gca()

for face in analysis.faces:
    bound = face.face_rectangle
  
    rect = patches.Rectangle((bound.left, bound.top),
                     bound.width,
                     bound.height,
                     linewidth=4,
                     edgecolor='red',
                     fill = False)

    ax.add_patch(rect)

plt.show()

print(f'{len(analysis.faces)}人の人が写っています')

for face in analysis.faces:
    print(f'{"男性" if face.gender == "Male" else "女性"} {face.age}歳')

> 上では if - else を 1行で記述しています。
>
> ```python
> 条件が成立する時の処理 if 条件式 else 条件が成立しないときの処理
> ```
>
> ソースコードが短くなるので、慣れると書くのも読むのも簡単になります。  
> 今は、このような書き方もできるのかと思うくらいでもかまいません。

---

## 2. 自然言語処理

In [ ]:
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

In [ ]:
review_folder = os.path.join('data', 'text')

reviews = []
for filename in os.listdir(review_folder):
    if filename.endswith(".txt"):
        text = open(os.path.join(review_folder, filename), 'r', encoding='UTF-8').read()
        review = {"id": filename, "text": text}
        reviews.append(review)

In [ ]:
for review in reviews:
    print(f'--- {review["id"]} ---\n{review["text"]}')
    print()

In [ ]:
# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

In [ ]:
# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

In [ ]:
# print detected language details for each review
for review_num in range(len(reviews)):
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(f'{reviews[review_num]["id"]}: {lang.name} ({(lang.score * 100):.1f}%)')
    
    reviews[review_num] = {"id": reviews[review_num]["id"], "text": reviews[review_num]["text"], "language": lang.iso6391_name}

In [ ]:
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

In [ ]:
# print key phrases for each review
for review_num in range(len(reviews)):
    print(f'{reviews[review_num]["id"]} のキーフレーズ:')

    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
        
    print()

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

In [ ]:
for review_num in range(len(reviews)):
    sentiment_score = sentiment_analysis.documents[review_num].score
    sentiment = "ネガティブ" if sentiment_score < 0.5 else "ポジティブ"
    print(f'{sentiment_analysis.documents[review_num].id}: {sentiment} ({(sentiment_score * 100):.1f}%)')